In [1]:
# Title: Lambdaa function for data wrangling
# Date: 2019-03-28 3:00
# Tags: python
# Slug: Code Snippet

You can throw compute at a problem, but it's probably best to write better code in general.

Zen of Python.
"Sparse is better than dense." (Sparse in a diff sense, but still)
Code that works is code that works.
Simple is better than complex.

# Example 1 - Don't iterate through every row

In [ ]:
%%time 
seed_and_names["final_four"] = (seed_and_names[["team_id","season"]].
                                apply(lambda seed_and_names: 
                                      is_final_four(seed_and_names["team_id"],
                                                    seed_and_names["season"]),axis=1))

In [10]:
print(f'CPU times: user 3.86 s, sys: 39.5 ms, total: 3.9 s \n Wall time: 3.96 s') # on 1000s

CPU times: user 3.86 s, sys: 39.5 ms, total: 3.9 s 
 Wall time: 3.96 s


# Vectorized solutions

In [ ]:
%%time 
seed_and_names["final_four"]=np.zeros(len(seed_and_names)) # zeros

final_four_list = (list(zip(mm.query("final_four==1").season,mm.query("final_four==1").winning_team_id))+
     list(zip(mm.query("final_four==1").season,mm.query("final_four==1").losing_team_id)))

# fill in teams with 1 if final_four team
for season, team_id in final_four_list:
    seed_and_names["final_four"]+=np.where((seed_and_names.season==season) & (seed_and_names.team_id==team_id),1,0)

In [11]:
print(f'CPU times: user 112 ms, sys: 4.93 ms, total: 117 ms \n Wall time: 116 ms')

CPU times: user 112 ms, sys: 4.93 ms, total: 117 ms 
 Wall time: 116 ms


In [17]:
f' Using  {str(round(3.96*1000/116,2))}"x faster"

'34.14x faster'

In [ ]:
# on 1000 rows, but more calculations
# but I guess it's doing a lot of work to sum up

In [18]:
print(f' CPU times: user 3min 15s, sys: 18.5 s, total: 3min 34s \n Wall time: 2min 15s')

 CPU times: user 3min 15s, sys: 18.5 s, total: 3min 34s 
 Wall time: 2min 15s


Why vectorized solutions are best , no pandas in for loops, don't make your soup too big! wow from 30 mins to 5 mins

In [ ]:
Dask

In [ ]:
# bad
# Let's write a function to automate this for any team and season
# Note, ran into error with Wichita St. since they were undefeated, 
# need to add try except clause, to avoid indexing issues

def points_for(team_id, season):
    '''This function generates the team's total points scored in a season.
    points_for(1102,2003) >>> 1603'''
    # points from winning games
    try: 
        i = regular.groupby(["season","winning_team_id"]).sum().query(f'winning_team_id=={team_id} & season=={season}').values[0][1]
    except:
        i = 0
    # points from losing games
    try:
        j = regular.groupby(["season","losing_team_id"]).sum().query(f'losing_team_id=={team_id} & season=={season}').values[0][3]
    except:
        j = 0
    return i+j

def points_against(team_id,season):
    '''This function generates the team's points against in a season.
    points_against(1102,2003) >> 1596'''
    # points allowed from winning games
    try:
        i = regular.groupby(["season","winning_team_id"]).sum().query(f'winning_team_id=={team_id} & season=={season}').values[0][3]
    except:
        i = 0
    # points allowed from losing games
    try:
        j = regular.groupby(["season","losing_team_id"]).sum().query(f'losing_team_id=={team_id} & season=={season}').values[0][2]
    except:
        j = 0
    return i+j

In [ ]:
# 2 mins 30 seconds 
%%time 

seed_and_names["points_for"] = (seed_and_names[["team_id","season"]].
                                apply(lambda seed_and_names: 
                                      points_for(seed_and_names["team_id"],
                                                    seed_and_names["season"]),axis=1))

seed_and_names["points_against"] = (seed_and_names[["team_id","season"]].
                                    apply(lambda seed_and_names: 
                                          points_against(seed_and_names["team_id"],
                                                         seed_and_names["season"]),axis=1))